In [1]:
!pip install tensorflow

In [2]:
# Demo Red Neuronal Convolucional (CNN)
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
# Crear el modelo
model = Sequential()
# Añadir la capa convolucional
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(64, 64, 3)))
# Añadir una capa de pooling
model.add(MaxPooling2D(pool_size=(2, 2)))
# Aplanar el resultado para las capas densas
model.add(Flatten())
# Añadir una capa densa (fully connected)
model.add(Dense(units=128, activation='relu'))
# Capa de salida
model.add(Dense(units=1, activation='sigmoid'))
# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Resumen del modelo
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 31, 31, 32)        0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 30752)             0         
                                                                 
 dense (Dense)               (None, 128)               3936384   
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 3937409 (15.02 MB)
Trainable params: 3937409 (15.02 MB)
Non-trainable params: 0 (0.00 Byte)
________________

Preprocesamiento de imagenes

In [4]:
# Preprocesamiento de imágenes
def cargar_y_preprocesar_imagen(ruta_imagen, tamano=(64, 64)):
  # Cargar la imagen
  imagen = Image.open(ruta_imagen).convert('RGB')
  imagen = imagen.resize(tamano)
  imagen = np.array(imagen)
  imagen = imagen / 255.0 # Normalización
  return imagen

In [12]:
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
from PIL import Image
import numpy as np

# Rutas a las carpetas de gatos y perros
ruta_gatos ='/content/drive/MyDrive/dataIA/dogs_vs_cats/Cat'
ruta_perros ='/content/drive/MyDrive/dataIA/dogs_vs_cats/Dog'
# Cargar y preprocesar imágenes
imagenes = []
etiquetas = []

# Cargar gatos
for archivo in os.listdir(ruta_gatos):
  ruta_completa = os.path.join(ruta_gatos, archivo)

  # Lista de extensiones permitidas
  extensiones_permitidas = ['.jpg', '.jpeg', '.png', '.bmp']
  _, extension = os.path.splitext(archivo)

  # otra opción podría ser ruta_imagen[-4:].lower() para obtener la extensión pero mejor así.
  # Comprobar que la extensión es una de las permitidas
  if extension.lower() in extensiones_permitidas:
      try:
          img = cargar_y_preprocesar_imagen(ruta_completa)
          imagenes.append(img)
          etiquetas.append(0)  # 0 para gatos
      except Exception as e:
          print(f"No se pudo cargar la imagen {archivo}. Error: {str(e)}")
  else:
      continue

In [17]:
# Cargar perros
for archivo in os.listdir(ruta_perros):
  ruta_completa = os.path.join(ruta_perros, archivo)
  # Lista de extensiones permitidas
  extensiones_permitidas = ['.jpg', '.jpeg', '.png', '.bmp']
  _, extension = os.path.splitext(archivo)
  # otra opción podría ser ruta_imagen[-4:].lower() para obtener la extensión pero mejor así.
  # Comprobar que la extensión es una de las permitidas
  if (extension.lower() in extensiones_permitidas):
    try:
      img = cargar_y_preprocesar_imagen(ruta_completa)
      imagenes.append(img)
      etiquetas.append(1) # 1 para perros
    except:
      print(f"No se pudo cargar la imagen {archivo}")
  else:
    continue

In [20]:
from sklearn.model_selection import train_test_split

# Convertir listas a arrays de Numpy
imagenes = np.array(imagenes)
etiquetas = np.array(etiquetas)
X_train, X_test, y_train, y_test = train_test_split(imagenes, etiquetas, test_size=0.2, random_state=42)

# Construcción del modelo
modelo = Sequential([
  Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
  MaxPooling2D(2, 2),
  Conv2D(64, (3, 3), activation='relu'),
  MaxPooling2D(2, 2),
  Flatten(),
  Dense(128, activation='relu'),
  Dense(1, activation='sigmoid')
  ])

# Compilación del modelo
modelo.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [21]:
# Entrenamiento del modelo
modelo.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))
# Evaluar el modelo
evaluacion = modelo.evaluate(X_test, y_test)
print(f"Test Accuracy: {evaluacion[1] * 100}%")
modelo.save('modelo_cats_vs_dogs.keras') # Guarda el modelo en un archivo .keras

Epoch 1/10
367/367 [==============================] - 80s 213ms/step - loss: 0.4248 - accuracy: 0.8509 - val_loss: 0.4038 - val_accuracy: 0.8525
Epoch 2/10
367/367 [==============================] - 70s 191ms/step - loss: 0.3699 - accuracy: 0.8516 - val_loss: 0.3628 - val_accuracy: 0.8525
Epoch 3/10
367/367 [==============================] - 71s 194ms/step - loss: 0.3423 - accuracy: 0.8516 - val_loss: 0.3508 - val_accuracy: 0.8525
Epoch 4/10
367/367 [==============================] - 74s 202ms/step - loss: 0.3154 - accuracy: 0.8625 - val_loss: 0.3416 - val_accuracy: 0.8641
Epoch 5/10
367/367 [==============================] - 72s 197ms/step - loss: 0.2813 - accuracy: 0.8869 - val_loss: 0.3344 - val_accuracy: 0.8743
Epoch 6/10
367/367 [==============================] - 75s 205ms/step - loss: 0.2492 - accuracy: 0.8996 - val_loss: 0.3463 - val_accuracy: 0.8637
Epoch 7/10
367/367 [==============================] - 72s 197ms/step - loss: 0.2105 - accuracy: 0.9148 - val_loss: 0.3560 - val_ac

In [22]:
def predecir_imagen(ruta_imagen):
  modelo = load_model('modelo_cats_vs_dogs.keras')
  imagen = cargar_y_preprocesar_imagen(ruta_imagen)
  imagen = np.expand_dims(imagen, axis=0)
  prediccion = modelo.predict(imagen)
  if prediccion[0][0] > 0.5:
    print("Es un perro")
  else:
    print("Es un gato")


In [24]:
!pip install utils

  Preparing metadata (setup.py) ... done
  Created wheel for utils: filename=utils-1.0.2-py2.py3-none-any.whl size=13905 sha256=a8f8e1f976b5617ac61227c4538f538be7525b5189b87f8efdaece5a681e701d
  Stored in directory: /root/.cache/pip/wheels/b8/39/f5/9d0ca31dba85773ececf0a7f5469f18810e1c8a8ed9da28ca7
Successfully built utils


In [30]:
from keras.models import load_model

modelo = load_model('modelo_cats_vs_dogs.keras')
predecir_imagen('/content/drive/MyDrive/dataIA/dogs_vs_cats/predict/perro1.jpg')
predecir_imagen('/content/drive/MyDrive/dataIA/dogs_vs_cats/predict/perro2.jpg')
predecir_imagen('/content/drive/MyDrive/dataIA/dogs_vs_cats/predict/gato1.jpg')

1/1 [==============================] - 0s 87ms/step
Es un perro


1/1 [==============================] - 0s 84ms/step
Es un gato
1/1 [==============================] - 0s 86ms/step
Es un gato
